# Activity 9

This activity tests your skills
in approaching an engineering problem
as solving a system of nonlinear algebraic equations.

## Scenario

Referring to [the diagram](./act-09.png),
a device has an active power consumption $P$
when operating at a lagging power factor $\Phi$.
The requisite power is conveyed by the
resistance $R$,
inductance $L$
and the capacitor $C$,
from a source
configured to provide a steady RMS voltage $U$
at a steady frequency $F$.

You are to solve for the following quantities.
- $V$, the RMS operating voltage of the device
- $\delta_{\text{l}}$,
  the phase angle of the voltage phasor at which the device receives power
- $\delta_{\text{s}}$
  the phase angle of the voltage phasor provided by the source

## Modelling

Cast the original task as
solving a system of nonlinear algebraic equations
in three unknowns
(namely, $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$).
Introduce a vector $\boldsymbol{x}$ 
to collect the unknowns in the order presented above,
so that the system of equations can be written in vector form as
$\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$.
Set the voltage phasor across the input terminals of the load
to have the reference phase angle.

## Core code

In [1]:
import math as mt
import numpy as np
import scipy as sp

Define a Python function
`f()` that takes

- one positional argument,
  `x`, representing $\boldsymbol{x}$;
  and
- four keyword arguments,
  `U`, `F`, `R`, `L`, `C`, `P`, and `Phi`
  representing $U$, $F$, $R$, $L$, $C$, $P$, and $\Phi$ respectively.

`f()` returns a NumPy array representing
$\boldsymbol{f}\!\left(\boldsymbol{x}\right)$.

Moreover, define a Python function `dfdx()`
that takes the same arguments as `f()`,
and returns a NumPy array representing
the Jacobian of $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ w.r.t. $\boldsymbol{x}$.


In [2]:
#guided by GoogleGeminiAI
def f(x, U, F, R, L, C, P, Phi):
    V = x[0]
    d_l = x[1]
    d_s = x[2]

    omega = 2 * np.pi * F
    X_l = omega * L
    
    # ### FIX 1: Added parentheses for correct order of operations
    X_C = 1 / (omega * C)

    # Q_net = Q_load - Q_cap
    # Note: Q_load is positive (lagging), Q_cap is V^2/Xc
    Q_load = P * np.tan(np.arccos(Phi))
    Q_net = Q_load - (V**2) / X_C

    # f1: Real Power Balance
    f1 = V**2 - U*V*np.cos(d_s - d_l) + (P*R + Q_net*X_l)
    
    # f2: Reactive Power Balance
    f2 = U*V*np.sin(d_s - d_l) - (P*X_l - Q_net*R)
    
    # f3: Reference Angle Constraint
    f3 = d_l - 0.0
    
    return np.array([f1, f2, f3])

def dfdx(x, U, F, R, L, C, P, Phi):
    V = x[0]
    d_l = x[1]
    d_s = x[2]
    omega = 2 * np.pi * F

    # Pre-compute trig terms for cleaner code
    sin_val = np.sin(d_s - d_l)
    cos_val = np.cos(d_s - d_l)

    # Derivative of f1 wrt V
    # d(Q_net)/dV = -2 * V * omega * C
    # df1/dV term becomes: 2V + X_l * (-2*V*omega*C) = 2V(1 - omega^2*L*C)
    df1_dV = 2*V*(1 - (omega**2)*L*C) - U*cos_val
    
    # Derivative of f2 wrt V
    # df2/dV term becomes: - (-R * (-2*V*omega*C)) = - 2*V*R*omega*C
    # ### FIX 2: Changed sign from + to -
    df2_dV = U*sin_val - 2*V*R*omega*C

    return np.array([
        [df1_dV,             -U*V*sin_val,   U*V*sin_val],
        [df2_dV,             -U*V*cos_val,   U*V*cos_val],
        [0,                  1,              0]
    ])

In [3]:
help(sp.optimize.root)

Help on function root in module scipy.optimize._root:

root(fun, x0, args=(), method='hybr', jac=None, tol=None, callback=None, options=None)
    Find a root of a vector function.

    Parameters
    ----------
    fun : callable
        A vector function to find a root of.
    x0 : ndarray
        Initial guess.
    args : tuple, optional
        Extra arguments passed to the objective function and its Jacobian.
    method : str, optional
        Type of solver. Should be one of

            - 'hybr'             :ref:`(see here) <optimize.root-hybr>`
            - 'lm'               :ref:`(see here) <optimize.root-lm>`
            - 'broyden1'         :ref:`(see here) <optimize.root-broyden1>`
            - 'broyden2'         :ref:`(see here) <optimize.root-broyden2>`
            - 'anderson'         :ref:`(see here) <optimize.root-anderson>`
            - 'linearmixing'     :ref:`(see here) <optimize.root-linearmixing>`
            - 'diagbroyden'      :ref:`(see here) <optimize.root

## Assessment

Every time you solve for $\boldsymbol{x}$,
- define Python variables
  `U`, `F`, `R`, `L`, `C`, `P`, and `PHI`
  for storing the values of parameters
  $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$,
  respectively;
- define a Python variable `x_ini`
  for storing your initial guesses of the variables;
- use [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html),
  particularly its `args` keyword argument;
- define a Python variable `x_out` to store $\boldsymbol{x}$;
- verify if $\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$ is sufficiently satisfied;
- print the value of $V$ in kilovolts;
  and
- print the values of $\delta_{\text{l}}$ and $\delta_{\text{s}}$ in degrees.

### Task 1

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
40 kilowatts, 0.87.

In [4]:
U = 13000 # load voltage guess should be very close to source voltage
val_1 = (13800, 60, 0.21, 5.570*10**-3, 1.560*10**-3, 40000, 0.87)
x_ini1 = np.array([U, 0.1, 0.1]) #initial guess
x_out1 = sp.optimize.root(f, x_ini1, args=val_1, jac=dfdx)

solution = x_out1.x
print(f"Voltage: {x_out1.x[0]*10**-3} kV")
print(f"Load Angle: {mt.degrees(x_out1.x[1])}°")
print(f"Source Angle: {mt.degrees(x_out1.x[2])}°")

f(x_out1.x, *val_1)#must be close to 0


Voltage: 0.007030744188677298 kV
Load Angle: -6.697012116770912e-23°
Source Angle: 54.755296504311126°


array([ 5.37031156e-07,  2.93875928e-07, -1.16884911e-24])

In [5]:
print(x_out1)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 5.370e-07  2.939e-07 -1.169e-24]
       x: [ 7.031e+00 -1.169e-24  9.557e-01]
  method: hybr
    nfev: 25
    njev: 3
    fjac: [[-3.771e-01  9.262e-01 -5.404e-27]
           [-9.262e-01 -3.771e-01  1.279e-05]
           [-1.185e-05 -4.823e-06 -1.000e+00]]
       r: [ 1.380e+04  1.492e+01  2.225e+04  7.819e+04 -9.688e+04
           -1.239e+00]
     qtf: [-4.327e-05  3.784e-04  4.839e-09]


### Task 2

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
32 kilowatts, 0.97.

In [6]:
U = 13000 # load voltage guess should be very close to source voltage
val_2 = (13800, 50, 0.17, 6.685*10**-3, 1.872*10**-3, 32000, 0.97)
x_ini2 = np.array([U, 0.1, 0.1]) #initial guess
x_out2 = sp.optimize.root(f, x_ini2, args=val_2, jac=dfdx)

solution = x_out2.x
print(f"Voltage: {x_out2.x[0]*10**-3} kV")
print(f"Load Angle: {mt.degrees(x_out2.x[1])}°")
print(f"Source Angle: {mt.degrees(x_out2.x[2])}°")

f(x_out2.x, *val_2)#must be close to 0

Voltage: 0.005036996924008476 kV
Load Angle: 9.519681342282928e-25°
Source Angle: 71.30766181744471°


array([1.18852768e-08, 4.38012648e-09, 1.66149783e-26])

In [7]:
print(x_out2)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 1.189e-08  4.380e-09  1.661e-26]
       x: [ 5.037e+00  1.661e-26  1.245e+00]
  method: hybr
    nfev: 24
    njev: 3
    fjac: [[-3.706e-01  9.288e-01  8.086e-26]
           [-9.288e-01 -3.706e-01  3.347e-05]
           [-3.109e-05 -1.240e-05 -1.000e+00]]
       r: [ 1.380e+04 -9.707e-01 -3.712e+03  2.988e+04 -6.954e+04
           -2.328e+00]
     qtf: [-3.467e-07 -5.824e-06 -1.949e-10]


### Task 3

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
214 kilowatts, 0.87.

In [8]:
U = 32000 # load voltage guess should be very close to source voltage
val_3 = (32500, 60, 0.21, 5.570*10**-3, 1.560*10**-3, 214000, 0.87)
x_ini3 = np.array([U, 0.1, 0.1]) #initial guess
x_out3 = sp.optimize.root(f, x_ini3, args=val_3, jac=dfdx)

solution = x_out3.x
print(f"Voltage: {x_out3.x[0]*10**-3} kV")
print(f"Load Angle: {mt.degrees(x_out3.x[1])}°")
print(f"Source Angle: {mt.degrees(x_out3.x[2])}°")

f(x_out3.x, *val_3)#must be close to 0

Voltage: 0.01597169748937318 kV
Load Angle: -6.53859818713548e-25°
Source Angle: 54.75502456805616°


array([ 2.75322236e-08,  1.33295543e-08, -1.14120067e-26])

In [9]:
print(x_out3)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 2.753e-08  1.333e-08 -1.141e-26]
       x: [ 1.597e+01 -1.141e-26  9.557e-01]
  method: hybr
    nfev: 26
    njev: 3
    fjac: [[-2.843e-01  9.587e-01 -1.296e-27]
           [-9.587e-01 -2.843e-01  2.252e-06]
           [-2.159e-06 -6.402e-07 -1.000e+00]]
       r: [ 3.250e+04  9.487e+00  1.712e+05  4.441e+05 -5.191e+05
           -1.169e+00]
     qtf: [-5.581e-05  3.386e-04  7.624e-10]


### Task 4

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
173 kilowatts, 0.97.

In [10]:
U = 32000 # load voltage guess should be very close to source voltage
val_4 = (32500, 50, 0.17, 6.685*10**-3, 1.872*10**-3, 173000, 0.97)
x_ini4 = np.array([U, 0.1, 0.1]) #initial guess
x_out4 = sp.optimize.root(f, x_ini4, args=val_4, jac=dfdx)

solution = x_out4.x
print(f"Voltage: {x_out4.x[0]*10**-3} kV")
print(f"Load Angle: {mt.degrees(x_out4.x[1])}°")
print(f"Source Angle: {mt.degrees(x_out4.x[2])}°")

f(x_out4.x, *val_4)#must be close to 0

Voltage: 0.011562811836882713 kV
Load Angle: -3.3586169377554844e-24°
Source Angle: 71.30752722447465°


array([ 4.74974513e-08,  8.55652615e-09, -5.86189239e-26])

In [11]:
print(x_out4)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 4.750e-08  8.557e-09 -5.862e-26]
       x: [ 1.156e+01 -5.862e-26  1.245e+00]
  method: hybr
    nfev: 31
    njev: 3
    fjac: [[-2.878e-02  9.996e-01 -8.106e-28]
           [-9.996e-01 -2.878e-02  5.518e-06]
           [-5.515e-06 -1.588e-07 -1.000e+00]]
       r: [ 3.250e+04  2.799e+01  1.126e+05  1.812e+05 -3.760e+05
           -2.074e+00]
     qtf: [ 4.578e-05 -3.072e-04 -1.695e-09]


### Task 5

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
69.0 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
421 kilowatts, 0.87.

In [12]:
U = 68000 # load voltage guess should be very close to source voltage
val_5 = (69000, 60, 0.21, 5.570*10**-3, 1.560*10**-3, 421000, 0.87)
x_ini5 = np.array([U, 0.1, 0.1]) #initial guess
x_out5 = sp.optimize.root(f, x_ini5, args=val_5, jac=dfdx)

solution = x_out5.x
print(f"Voltage: {x_out5.x[0]*10**-3} kV")
print(f"Load Angle: {mt.degrees(x_out5.x[1])}°")
print(f"Source Angle: {mt.degrees(x_out5.x[2])}°")

f(x_out5.x, *val_5)#must be close to 0

Voltage: 0.014799868078985642 kV
Load Angle: -1.338660478293607e-23°
Source Angle: 54.75084926091168°


array([-1.20908953e-06, -2.64390837e-06, -2.33640329e-25])

In [13]:
print(x_out5)

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.209e-06 -2.644e-06 -2.336e-25]
       x: [ 1.480e+01 -2.336e-25  9.556e-01]
  method: hybr
    nfev: 32
    njev: 2
    fjac: [[-9.952e-01 -9.764e-02 -1.179e-24]
           [ 9.764e-02 -9.952e-01  1.674e-06]
           [ 1.635e-07 -1.666e-06 -1.000e+00]]
       r: [ 6.900e+04  9.024e+01 -1.189e+06  5.973e+05 -1.024e+06
           -1.714e+00]
     qtf: [-2.558e-04 -4.471e-04 -7.486e-10]


### Task 6

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
317 kilowatts, 0.97.

In [14]:
U = 32000 # load voltage guess should be very close to source voltage
val_6 = (32500, 50, 0.17, 6.685*10**-3, 1.872*10**-3, 317000, 0.97)
x_ini6 = np.array([U, 0.1, 0.1]) #initial guess
x_out6 = sp.optimize.root(f, x_ini6, args=val_6, jac=dfdx)

solution = x_out6.x
print(f"Voltage: {x_out6.x[0]*10**-3} kV")
print(f"Load Angle: {mt.degrees(x_out6.x[1])}°")
print(f"Source Angle: {mt.degrees(x_out6.x[2])}°")

f(x_out6.x, *val_6)#must be close to 0

Voltage: 0.021187470350113462 kV
Load Angle: -3.9091475455204813e-25°
Source Angle: 71.3118498598708°


array([-1.04773790e-08, -3.25962901e-09, -6.82274956e-27])

In [15]:
print(x_out6)

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.048e-08 -3.260e-09 -6.823e-27]
       x: [ 2.119e+01 -6.823e-27  1.245e+00]
  method: hybr
    nfev: 24
    njev: 3
    fjac: [[-2.777e-01  9.607e-01  2.584e-27]
           [-9.607e-01 -2.777e-01  3.157e-06]
           [-3.032e-06 -8.767e-07 -1.000e+00]]
       r: [ 3.250e+04 -2.692e+01  3.080e+04  3.168e+05 -6.887e+05
           -2.174e+00]
     qtf: [-4.149e-07  9.289e-06  2.932e-11]


## Instructions

Do not use any library or module other than those pre-imported.

### Scoring

In each Task,
- a correct use of
  [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)
  merits two (2) points;
- obtaining a correct $V$ merits two (3) points;
- obtaining a correct $\delta_{\text{l}}$ merits two (3) points;
- obtaining a correct $\delta_{\text{s}}$ merits two (3) points;
  and
- obtaining $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ close enough to $\boldsymbol{0}$ merits six (9) points.


Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 120 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-09_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-09_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-12-11*